In [396]:
import pandas as pd
import numpy as np
from datetime import date, timedelta

from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger

##########################################################################

train_out = pd.read_pickle('../data/train_1s.p')


In [408]:
test_out = pd.read_pickle('../data/test.p')

In [397]:
test_out = pd.read_pickle('../data/test.p').reset_index()

In [412]:
test_out.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3370464 entries, (1, 96995, 2017-08-16 00:00:00) to (54, 2134244, 2017-08-31 00:00:00)
Data columns (total 2 columns):
id             int64
onpromotion    bool
dtypes: bool(1), int64(1)
memory usage: 41.8 MB


In [399]:
test_wed = test_out.loc[(test_out['date'].dt.dayofweek == 2), ]


In [400]:
test_wed = pd.merge(test_wed, train_2017_wed_promo_s_i[['store_nbr', 'item_nbr']],
                          on = ('store_nbr', 'item_nbr'), how = 'inner')

In [401]:
test_wed.head(2)

,store_nbr,item_nbr,date,id,onpromotion
0,1,253103,2017-08-16,125497239,False
1,1,253103,2017-08-23,126971817,False


In [402]:



test_wed['onpromotion'] = False
test_wed = test_wed.rename(columns={'onpromotion': 'new_onpromotion'})
del test_wed['id']

## !!!! Stupid way to set onpromotion to False 
test_out_new = pd.merge(test_out, test_wed, on=('store_nbr', 'item_nbr', 'date'), how='left')
test_out_new['onpromotion'] = test_out_new.new_onpromotion.combine_first(test_out_new.onpromotion)

del test_out_new['new_onpromotion']


print( test_out.groupby('onpromotion').size()) 
print( test_out_new.groupby('onpromotion').size()) 

print( test_out.shape) 
print( test_out_new.shape) 

onpromotion
False    3171867
True      198597
dtype: int64
onpromotion
False    3172542
True      197922
dtype: int64
(3370464, 5)
(3370464, 5)


In [403]:
test_out_new.head(1)

,store_nbr,item_nbr,date,id,onpromotion
0,1,96995,2017-08-16,125497040,False


In [351]:
import pandas as pd
import numpy as np
from datetime import date, timedelta

from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger

##########################################################################

train_out = pd.read_pickle('../data/train_1s.p')


###############################
# Find store-items that is always on promo at Wed 

train_2017 = train_out.loc[train_out['date'] > date(2017,1,1),]

train_2017_wed = train_2017.loc[(train_2017['date'].dt.dayofweek == 2), ]
train_2017_wed.onpromotion =train_2017_wed.onpromotion.astype(int)

train_2017_wed_promo = train_2017_wed[["store_nbr", "item_nbr", "date", "onpromotion"]]\
                                      .groupby(['item_nbr','store_nbr'])\
                                      .agg({'onpromotion': 'sum', 'date':'count'})\
                                      .reset_index()

train_2017_wed_promo_s_i = train_2017_wed_promo.loc[
    train_2017_wed_promo['onpromotion'] == train_2017_wed_promo['date'],]

###############################
 
## It is very slow to use index.isin
## train_out[train_out.index.isin(train_2017_wed_a.index)]['onpromotion'] = False

# 
train_wed = train_out.loc[(train_out['date'].dt.dayofweek == 2), ]

train_wed = pd.merge(train_wed, train_2017_wed_promo_s_i[['store_nbr', 'item_nbr']],
                          on = ('store_nbr', 'item_nbr'), how = 'inner')


train_wed['onpromotion'] = False
train_wed = train_wed.rename(columns={'onpromotion': 'new_onpromotion'})
del train_wed['unit_sales']

## !!!! Stupid way to set onpromotion to False 
train_out_new = pd.merge(train_out, train_wed, on=('store_nbr', 'item_nbr', 'date'), how='left')
train_out_new['onpromotion'] = train_out_new.new_onpromotion.combine_first(train_out_new.onpromotion)

del train_out_new['new_onpromotion']


print( train_out.groupby('onpromotion').size()) 
print( train_out_new.groupby('onpromotion').size()) 

print( train_out.shape) 
print( train_out_new.shape) 



/home/zyp/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


onpromotion
False    1939280
True      138020
dtype: int64
onpromotion
False    1968859
True      109955
dtype: int64
(2562153, 5)
(2562153, 5)


In [341]:

train_2017 = train_out.loc[train_out['date'] > date(2017,1,1),]

train_2017_wed = train_2017.loc[(train_2017['date'].dt.dayofweek == 2), ]
train_2017_wed.onpromotion =train_2017_wed.onpromotion.astype(int)

train_2017_wed_promo = train_2017_wed[["store_nbr", "item_nbr", "date", "onpromotion"]]\
                                      .groupby(['item_nbr','store_nbr'])\
                                      .agg({'onpromotion': 'sum', 'date':'count'})\
                                      .reset_index()

train_2017_wed_promo_s_i = train_2017_wed_promo.loc[
    train_2017_wed_promo['onpromotion'] == train_2017_wed_promo['date'],]

/home/zyp/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [342]:
train_2017_wed_promo_s_i.head(2)

,item_nbr,store_nbr,onpromotion,date
184,253103,1,2,2
1469,1149069,1,21,21


In [343]:
train_wed = train_out.loc[(train_out['date'].dt.dayofweek == 2), ]

train_wed = pd.merge(train_wed, train_2017_wed_promo_s_i[['store_nbr', 'item_nbr']],
                          on = ('store_nbr', 'item_nbr'), how = 'inner')

## It is very slow to use index.isin
## train_out[train_out.index.isin(train_2017_wed_a.index)]['onpromotion'] = False


train_wed['onpromotion'] = False
train_wed = train_wed.rename(columns={'onpromotion': 'new_onpromotion'})
del train_wed['unit_sales']


In [349]:
train_out.groupby('onpromotion').size()

/home/zyp/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


AttributeError: 'DataFrame' object has no attribute 'new_onpromotion'

In [345]:
train_wed.groupby('new_onpromotion').size()

new_onpromotion
False    30091
dtype: int64

In [346]:
train_out_new = pd.merge(train_out, train_wed, on=('store_nbr', 'item_nbr', 'date'), how='left')

In [347]:
train_out_new.head(1)

,date,store_nbr,item_nbr,unit_sales,onpromotion,new_onpromotion
0,2013-01-02,1,103665,1.098612,NaN,NaN


In [348]:

train_out_new['onpromotion'] = train_out_new.new_onpromotion.combine_first(train_out_new.onpromotion)

del train_out_new['new_onpromotion']

In [339]:


train_out_new.groupby('onpromotion').size()


onpromotion
False    1968859
True      109955
dtype: int64

In [291]:
train_out.shape

(2562153, 5)

In [292]:
train_2017 = train_out.loc[train_out['date'] > date(2017,1,1),]

In [293]:
train_2017.shape

(446915, 5)

In [294]:
train_2017_wed = train_2017.loc[(train_2017['date'].dt.dayofweek == 2), ]

In [295]:
train_2017_wed.onpromotion =train_2017_wed.onpromotion.astype(int)

/home/zyp/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [296]:
train_2017_wed.shape

(68368, 5)

In [297]:
train_2017.head(2)

,date,store_nbr,item_nbr,unit_sales,onpromotion
2115238,2017-01-02,1,103520,0.693147,False
2115239,2017-01-02,1,105575,1.386294,False


In [298]:
train_2017_wed_promo = train_2017_wed[["store_nbr", "item_nbr", "date", "onpromotion"]]\
                                      .groupby(['item_nbr','store_nbr'])\
                                      .agg({'onpromotion': 'sum', 'date':'count'})\
                                      .reset_index()

In [299]:
train_2017_wed.head(1)

,date,store_nbr,item_nbr,unit_sales,onpromotion
2119013,2017-01-04,1,99197,0.693147,0


In [300]:
train_2017_wed_promo_s_i = train_2017_wed_promo.loc[
    train_2017_wed_promo['onpromotion'] == train_2017_wed_promo['date'],]

In [301]:
train_2017_wed_promo_s_i.shape

(277, 4)

In [302]:
train_2017_wed_promo_s_i.head(1)

,item_nbr,store_nbr,onpromotion,date
184,253103,1,2,2


In [303]:
train_2017_wed = pd.merge(train_2017_wed, train_2017_wed_promo_s_i[['store_nbr', 'item_nbr']],
                          on = ('store_nbr', 'item_nbr'), how = 'inner')

In [304]:
train_2017_wed['onpromotion'] = False
train_2017_wed = train_2017_wed.rename(columns={'onpromotion': 'new_onpromotion'})
del train_2017_wed['unit_sales']

In [266]:
train_2017_wed.head(1)

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2017-01-04,1,1149069,1.098612,False


In [263]:
train_out.groupby('onpromotion').size()

onpromotion
False    1939280
True      138020
dtype: int64

In [245]:
train_out.groupby('onpromotion').size()

train_out['onpromotion'] = train_out.onpromotion.combine_first(train_2017_wed.onpromotion)
train_out_new = pd.merge(train_out, train_2017_wed, on=('store_nbr', 'item_nbr', 'date'), how='left')
del train_out_new['new_onpromotion']

train_out.groupby('onpromotion').size()


In [282]:
  train_out_new['onpromotion'] = train_out_new.new_onpromotion.combine_first(train_out_new.onpromotion)

In [285]:
train_out_new.groupby('onpromotion').size()

onpromotion
False    1946292
True      131008
dtype: int64

In [286]:
train_out_new.head(1)

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2013-01-02,1,103665,1.098612,NaN


In [200]:
    t_new = pd.merge(df_prev, df_sub, on=['id'], how='left')
    t_new['unit_sales'] = t_new.unit_sales_y.combine_first(t_new.unit_sales_x)

KeyError: ('store_nbr', 'item_nbr')

In [193]:
train_2017_wed_a.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7012 entries, (1, 1149069, 2017-01-04 00:00:00) to (1, 1441511, 2017-08-09 00:00:00)
Data columns (total 2 columns):
unit_sales     7012 non-null float32
onpromotion    7012 non-null int64
dtypes: float32(1), int64(1)
memory usage: 112.2 KB


In [ ]:
df1[(~df1.col1.isin(common.col1))&(~df1.col2.isin(common.col2))]

In [74]:
train_2017[(train_2017.date.isin(train_2017_wed.date)) &
          (train_2017.date.isin(train_2017_wed.date))].shape

(68368, 5)

In [83]:
train_2017[(train_2017.date.isin(train_2017_wed.date))].shape

(68368, 5)

In [87]:
train_2017[(train_2017['store_nbr'].isin(train_2017_wed_promo['store_nbr'])) & 
          (train_2017['item_nbr'].isin(train_2017_wed_promo['item_nbr']))].shape

(446915, 5)

In [ ]:
(train_2017['store_nbr'].isin(train_2017_wed_promo['store_nbr'])) & 
          (train_2017['item_nbr'].isin(train_2017_wed_promo['item_nbr']))&

In [ ]:
&(train_2017['onpromotion'] == True)